# LOAD


In [1]:
!pip install category_encoders
! pip install keras
import pandas as pd
import category_encoders as ce
from category_encoders import BinaryEncoder

In [2]:
DATA_FOLDER = "../../DATA/"
train = pd.read_parquet(DATA_FOLDER + "/PROCESSING/TRAIN_SPLIT.parq")
test = pd.read_parquet(DATA_FOLDER + "/PROCESSING/TEST_SPLIT.parq")
val = pd.read_parquet(DATA_FOLDER + "/PROCESSING/VAL_SPLIT.parq")
COMP_TEST = pd.read_parquet(DATA_FOLDER + "/PROCESSING/PREPEARED_COMP_TEST.parq")

In [3]:
train

,attack_time,watcher_country,watcher_as_num,watcher_as_name,attacker_country,attacker_as_num,attacker_as_name,attack_type,watcher_uuid_enum,attacker_ip_enum,label,protocol,joined_countries,port_count,HAS_TCP?,HAS_UDP?,risky_ports,Index_Column
45,-0.352171,AU,-0.076617,Entity Data Pty Limited,CA,-0.341899,AMANAHA-NEW,spam,-0.402299,16557,1,http,CA:AU,1,False,False,False,33
95,1.292507,FR,-0.469632,OVH SAS,FI,-0.416613,Hetzner Online GmbH,spam,-0.761499,175622,0,http,FI:FR,1,False,False,True,72
108,-0.352171,US,-0.003973,PRIVATESYSTEMS,LU,-0.132298,PONYNET,spam,-0.699756,84916,0,http,LU:US,3,False,False,True,82
112,-0.352171,US,-0.090606,IMH-IAD,US,-0.549383,HRTC,scan,-0.092007,85069,0,http,US:US,0,False,False,False,86
124,0.705122,US,-0.160335,Hostinger International Limited,US,-0.466781,NETWORK-SOLUTIONS-HOSTING,scan,-0.174904,195272,0,http,US:US,20,False,False,True,97
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
20006979,1.057553,US,-0.333226,AS-30083-GO-DADDY-COM-LLC,US,-0.402688,DREAMHOST-AS,exploit,-0.446258,75769,1,http,US:US,2,False,False,True,29538194
20006996,0.705122,DE,-0.124927,Contabo GmbH,US,1.432369,Suisse Limited,bruteforce,-0.567593,153844,0,ssh,US:DE,0,False,False,False,29538204
20006998,-0.704602,US,-0.311877,LIQUIDWEB,IR,-0.059960,Pars Online PJS,spam,-0.433063,151482,0,http,IR:US,15,False,False,True,29538206
20007005,1.292507,US,3.302596,GO-DADDY-COM-LLC,NL,-0.174004,Global Layer B.V.,spam,0.621731,162497,0,http,NL:US,2,False,False,False,29538211


#BINARY ENCODING


In [4]:
X_train = train.loc[:, train.columns != 'label']
X_test = test.loc[:, test.columns != 'label']
X_val = val.loc[:, val.columns != 'label']
y_test = test.loc[:, 'label']
y_train = train.loc[:, 'label']
y_val = train.loc[:, 'label']
del train
del test
del val

In [5]:
X_train.columns

Index(['attack_time', 'watcher_country', 'watcher_as_num', 'watcher_as_name',
       'attacker_country', 'attacker_as_num', 'attacker_as_name',
       'attack_type', 'watcher_uuid_enum', 'attacker_ip_enum', 'protocol',
       'joined_countries', 'port_count', 'HAS_TCP?', 'HAS_UDP?', 'risky_ports',
       'Index_Column'],
      dtype='object')

In [6]:
X_train.info()

<class 'pandas.core.frame.DataFrame'>
Index: 2130397 entries, 45 to 20007010
Data columns (total 17 columns):
 #   Column             Dtype   
---  ------             -----   
 0   attack_time        float64 
 1   watcher_country    category
 2   watcher_as_num     float32 
 3   watcher_as_name    category
 4   attacker_country   category
 5   attacker_as_num    float32 
 6   attacker_as_name   category
 7   attack_type        category
 8   watcher_uuid_enum  float64 
 9   attacker_ip_enum   int64   
 10  protocol           category
 11  joined_countries   category
 12  port_count         int64   
 13  HAS_TCP?           bool    
 14  HAS_UDP?           bool    
 15  risky_ports        bool    
 16  Index_Column       int64   
dtypes: bool(3), category(7), float32(2), float64(2), int64(3)
memory usage: 145.0 MB


In [7]:
batch_size = 500000  # Choose an appropriate batch size
num_batches = len(X_train) // batch_size
target_features = [ 'watcher_country',  'watcher_as_name', 'attacker_country', 'attacker_as_name', 'attack_type', 'protocol', 'joined_countries']
dataframes = {"COMP_COLUMN" : COMP_TEST, "X_TEST" : X_test,  "X_VAL" : X_val}
for x in target_features:
    print(x)
    BinaryEncoder = ce.BinaryEncoder(cols=x)
    unique_values = X_train[x].unique()
    data = pd.DataFrame({x: unique_values})
    BinaryEncoder.fit(data)

    X_train_binary = pd.DataFrame()

    for i in range(num_batches):
        start = i * batch_size
        end = (i + 1) * batch_size
        batch_X = X_train.iloc[start:end]
        batch_y = y_train.iloc[start:end]
        batch_X_encoded = BinaryEncoder.transform(batch_X[x], batch_y)
        X_train_binary = pd.concat([X_train_binary, batch_X_encoded], ignore_index=False)


    remain = num_batches * batch_size
    batch_X_encoded = BinaryEncoder.transform(X_train[x].iloc[remain:], y_train.iloc[remain:])
    X_train_binary = pd.concat([X_train_binary, batch_X_encoded], ignore_index=False)
    X_train_binary.to_parquet(DATA_FOLDER + '/ENCODED/X_TRAIN_Binary_{}.parq'.format(x), index=True)
    del X_train_binary

    for dataframe in dataframes:
        dataframe_ = BinaryEncoder.transform(dataframes[dataframe][x])
        dataframe_.to_parquet(DATA_FOLDER + '/ENCODED/{}_Binary_{}.parq'.format(dataframe, x), index=True)
        del dataframe_



pd.DataFrame({'label': y_train}).to_parquet(DATA_FOLDER + '/ENCODED/y_train_Binary.parq', index=True)
pd.DataFrame({'label': y_test}).to_parquet(DATA_FOLDER + '/ENCODED/y_test_Binary.parq', index=True)
pd.DataFrame({'label': y_val}).to_parquet(DATA_FOLDER + '/ENCODED/y_train_val.parq', index=True)

watcher_country
watcher_as_name
attacker_country
attacker_as_name
attack_type
protocol
joined_countries
